In [1]:
# Импорт всех необходимых модулей для оптимизации
import sys
import os
from pathlib import Path

# Добавляем путь к модулям проекта
project_root = Path.cwd()
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Основные модули оптимизации
from optimization import (
    # Основные классы
    OptimizationExecutor,
    OptimizationReporter,
    
    # Хранилище результатов
    InMemoryResultsStore,
    OptimizationResult,
    ResultsStoreProtocol,
    
    # Пространство параметров
    ParameterDefinition,
    ParameterSpace,
    
    # Стратегии поиска
    SearchStrategy,
    GridSearchStrategy,
    RandomSearchStrategy,
)

# Дополнительные импорты для работы с данными и бэктестингом
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Optional, Mapping, Sequence
from dataclasses import dataclass
from datetime import datetime, UTC

# Импорты для работы с сигналами и индикаторами
from signals.base_signal import BaseSignal
from signals.signal_registry import SignalRegistry
from indicators.base_indicator import BaseIndicator
from indicators.indicator_registry import IndicatorRegistry

# Импорты для бэктестинга
from backtest.engine import BacktestEngine
from backtest.metrics import MetricsCalculator
from backtest.trade import Trade
from backtest.commission import CommissionCalculator

print("✅ Все модули оптимизации успешно импортированы!")
print(f"📁 Рабочая директория: {project_root}")
print(f"📦 Доступные стратегии: GridSearchStrategy, RandomSearchStrategy")
print(f"📊 Доступные хранилища: InMemoryResultsStore")
print(f"📈 Доступные репортеры: OptimizationReporter")


✅ Все модули оптимизации успешно импортированы!
📁 Рабочая директория: c:\Users\user\Documents\spreader_pro\code\trend_optimization
📦 Доступные стратегии: GridSearchStrategy, RandomSearchStrategy
📊 Доступные хранилища: InMemoryResultsStore
📈 Доступные репортеры: OptimizationReporter


In [4]:
# Ячейка 1: Импорты и настройка
import sys
sys.path.append('src')
from data import CSVLoader
import pandas as pd
import numpy as np

from lightweight_charts import JupyterChart
# Ячейка 2: Настройка данных
DATA_DIR = r"G:\My Drive\data_fut"
TICKER = "IMOEXF"
loader = CSVLoader(DATA_DIR, use_cache=True)

# Ячейка 3: Загрузка данных
start_date = '2025-09-17'
end_date = '2025-10-29'

df = loader.load(TICKER, timeframe=10, start_date=start_date, end_date=end_date)
print(f"Данные: {len(df)} баров, период: {df.index[0]} - {df.index[-1]}")

# Ячейка 4: Ваш анализ (например, Volatility Median)
# Здесь можете использовать функции из VM_base_optimize.py

Данные: 3071 баров, период: 2025-09-17 09:00:00 - 2025-10-28 17:40:00


In [5]:
# Простой способ - создание из ваших данных
your_params = {
    "KATR": [2, 3, 5, 8, 11, 15, 18, 25],
    "PerATR": [5, 10, 14, 20, 30], 
    "SMA": [1, 3, 5, 10, 20],
    "MinRA": [0.5, 1, 3, 5, 7, 10],
    "FlATR": [0, 1],
    "FlHL": [0, 1]
}

# Преобразование в ParameterSpace
definitions = []
for name, values in your_params.items():
    definitions.append(ParameterDefinition(
        name=name.lower(),  # приводим к нижнему регистру
        values=values
    ))

parameter_space = ParameterSpace.from_definitions(definitions)

In [11]:
parameter_space

ParameterSpace(parameters={'katr': ParameterDefinition(name='katr', description='', values=[2, 3, 5, 8, 11, 15, 18, 25], lower_bound=None, upper_bound=None, step=None, is_required=True), 'peratr': ParameterDefinition(name='peratr', description='', values=[5, 10, 14, 20, 30], lower_bound=None, upper_bound=None, step=None, is_required=True), 'sma': ParameterDefinition(name='sma', description='', values=[1, 3, 5, 10, 20], lower_bound=None, upper_bound=None, step=None, is_required=True), 'minra': ParameterDefinition(name='minra', description='', values=[0.5, 1, 3, 5, 7, 10], lower_bound=None, upper_bound=None, step=None, is_required=True), 'flatr': ParameterDefinition(name='flatr', description='', values=[0, 1], lower_bound=None, upper_bound=None, step=None, is_required=True), 'flhl': ParameterDefinition(name='flhl', description='', values=[0, 1], lower_bound=None, upper_bound=None, step=None, is_required=True)}, validators=[])

In [6]:
# спящая ячейка для отображения графика
df_chart = df.copy()
df_chart = df_chart.rename(columns={
    'OPEN': 'open',
    'HIGH': 'high',
    'LOW': 'low',
    'CLOSE': 'close',
    'VOL': 'volume'
})

print(f"Всего данных для отображения: {len(df_chart)} баров")

# Вариант 1: Отобразить все данные (может быть медленно)
chart = JupyterChart()
chart.set(df_chart)
chart.load()

Всего данных для отображения: 3071 баров


In [7]:
vm_ind = IndicatorRegistry.get("VolatilityMedian")
ind_params = vm_ind.get_param_grid()

In [9]:
vm = IndicatorRegistry.get('VolatilityMedian')
vm_params = dict(KATR=6, MinRA=0, PerATR=30, SMA=3)
indicator = vm.calculate(df, **vm_params).ffill()
indicator.name = 'VolatilityMedian'

In [10]:
naklon = SignalRegistry.get('SlopeSignal')
signals_naklon = naklon.generate(df, indicator, threshold=0.0001, absolute=False)

deviation = SignalRegistry.get('DeviationSignal')
signals = deviation.generate(df, indicator, offset=12, use_hl=True)
# signals = signals_naklon
signals.name = 'DeviationSignal'

signals.value_counts()

DeviationSignal
-1.0    2059
 1.0     980
 0.0      32
Name: count, dtype: int64

In [ ]:
# from signals import SignalRegistry

# slope = SignalRegistry.get('SlopeSignal')
# signals = slope.generate(df, indicator, threshold=0.0001, absolute=False)

# signals.name = 'SlopeSignal'

In [11]:
from backtest import BacktestEngine
engine = BacktestEngine(initial_capital=100_000.0)
result = engine.run(df, signals)
equity = result['equity']

In [12]:

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool



output_notebook()
p = figure(title='Equity (Bokeh)', x_axis_type='datetime', width=900, height=420, tools='pan,wheel_zoom,box_zoom,reset,save')
p.line(equity.index, equity.values, line_width=2, color='#2962FF', legend_label='Equity')
p.add_tools(HoverTool(tooltips=[('time', '@x{%F %T}'), ('equity', '@y')], formatters={'@x': 'datetime'}))
p.legend.location = 'top_left'
p.grid.grid_line_alpha = 0.25
show(p)


Loading BokehJS ...